In [1]:
import pandas as pd
import pandas_datareader.data as web
import math as mt
import numpy as np
import yfinance as yf

In [2]:
historicos = pd.read_csv('Lab_1/NAFTRAC_20220729.csv', skiprows=2)
historicos.head(5)

,Ticker,Nombre,Clase de activo,Peso (%),Precio,Acciones,Valor de mercado,Valor nominal,Sector,Unnamed: 9,ISIN,Bolsa de valores,Ubicación,Moneda,Mercado de divisas,Tipo de Cambio
0,AMXL,AMERICA MOVIL L,Equity,14.94,19.40,"429,379,400.00","8,329,960,360.00","8,329,960,360.00",Comunicación,2667470,MXP001691213,Bolsa Mexicana De Valores,México,MXN,MXN,1.0
1,WALMEX*,WALMART DE MEXICO V,Equity,12.88,73.84,"97,247,481.00","7,180,753,997.04","7,180,753,997.04",Productos básicos de consumo,BW1YVH8,MX01WA000038,Bolsa Mexicana De Valores,México,MXN,MXN,1.0
2,GFNORTEO,GPO FINANCE BANORTE,Equity,9.79,115.99,"47,068,861.00","5,459,517,187.39","5,459,517,187.39",Financieros,2421041,MXP370711014,Bolsa Mexicana De Valores,México,MXN,MXN,1.0
3,GMEXICOB,GRUPO MEXICO B,Equity,8.65,80.60,"59,802,565.00","4,820,086,739.00","4,820,086,739.00",Materiales,2643674,MXP370841019,Bolsa Mexicana De Valores,México,MXN,MXN,1.0
4,FEMSAUBD,FOMENTO ECONOMICO MEXICANO,Equity,8.04,127.03,"35,278,535.00","4,481,432,301.05","4,481,432,301.05",Productos básicos de consumo,2242059,MXP320321310,Bolsa Mexicana De Valores,México,MXN,MXN,1.0


In [3]:
filenames= ['NAFTRAC_20200131.csv','NAFTRAC_20200228.csv','NAFTRAC_20200331.csv','NAFTRAC_20200430.csv','NAFTRAC_20200529.csv','NAFTRAC_20200630.csv','NAFTRAC_20200731.csv','NAFTRAC_20200831.csv'
        ,'NAFTRAC_20200930.csv','NAFTRAC_20201030.csv','NAFTRAC_20201130.csv','NAFTRAC_20201231.csv','NAFTRAC_20210129.csv','NAFTRAC_20210226.csv','NAFTRAC_20210331.csv','NAFTRAC_20210331.csv'
        ,'NAFTRAC_20210430.csv','NAFTRAC_20210531.csv','NAFTRAC_20210630.csv','NAFTRAC_20210730.csv','NAFTRAC_20210831.csv','NAFTRAC_20210930.csv','NAFTRAC_20211026.csv','NAFTRAC_20211130.csv'
        ,'NAFTRAC_20211231.csv','NAFTRAC_20220126.csv','NAFTRAC_20220228.csv','NAFTRAC_20220331.csv','NAFTRAC_20220429.csv','NAFTRAC_20220531.csv','NAFTRAC_20220630.csv','NAFTRAC_20220729.csv']


In [24]:
loc = r'Lab_1/'
fullloc = [path + filenames[i] for i in range(0, len(filenames))]

In [25]:
df= [pd.read_csv(i,skiprows=2) for i in fullloc]
df=[df[i].dropna() for i in range(0,len(df))]
ticker=df[0]["Ticker"].reset_index(drop=True)
ticker=[ticker[i]+".MX" for i in range(0,len(ticker))]
ticker=[ticker[i].replace("*","")for i in range(0,len(ticker))]
ticker=[ticker[i].replace("GFREGIOO","RA") for i in range (0,len(ticker))]
ticker[34]="LIVEPOLC-1.MX"
ticker.remove("LIVEPOLC.1.MX")
ticker.remove("BSMXB.MX")
ticker.remove("KOFUBL.MX")


In [6]:
#data = pd.DataFrame(yf.download(ticker['Ticker'].unique().tolist(), start="2020-01-31", end="2022-07-30"))

In [7]:
def get_adj_closes(tickers, start_date=None, end_date=None):
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    closes = closes['Adj Close']
    closes.sort_index(inplace=True)
    return closes

In [8]:
# Inversión Pasiva
prices_M = pd.DataFrame(get_adj_closes(tickers=ticker, start_date="2020-01-31", end_date="2022-07-29")).dropna()
prices_M = prices_M.groupby(pd.Grouper(freq="M")).last()


In [9]:
#Inversion Activa
prices_A = pd.DataFrame(get_adj_closes(tickers=ticker, start_date="2020-01-31", end_date="2022-07-29")).dropna()
prices_A = prices_A.groupby(pd.Grouper(freq="M")).last()


In [26]:
#Capital Inicial
cap_i=1000000

#Primera fecha
FirstDF = df
passive_m1= df[0].reset_index(drop=True)

#Peso ponderado
peso_pond= passive_m1["Peso (%)"]/100

#Capital Invertido en T=0
peso_din= cap_i * peso_pond

In [28]:
acciones=prices_M.iloc[0, :]
#Cantidad de títulos por activo
titulos = [peso_din[i] / acciones[i] for i in range(0, len(acciones))]
n_titulos=[mt.trunc(titulos[i]) for i in range (0, len(titulos))]


In [29]:
#comisión para el portafolio
fee= [sum((n_titulos[i] * acciones) *0.00125) for i in range(0,len(acciones))]

#Precio mensual por la cantidad de titulos
value_i= sum(n_titulos*acciones)
#cantidad de efectivo  
cash= cap_i - value_i - sum(fee)


In [30]:
#Valor del portafolio al primer mes
pvalue_m1= value_i + cash 

#Valor del portafolio en el resto de los meses de inversión
pvalue_mr=[sum(prices_M.iloc[i, :] * titulos) for i in range (len(prices_M[0:]))]

In [47]:
df_pasiva=pd.DataFrame({'Timestamp': prices_M.index[0:], 'capital':pvalue_mr[0:]})
rend=[(pvalue_mr[i+1]/pvalue_mr[i])-1  for i in range(0,len (pvalue_mr)-1)]
rend.insert(0,0)
rend_acum= [(pvalue_mr[i]/pvalue_m1) for i in range(0, len(pvalue_mr))]
df_pasiva["Rend"] = rend
df_pasiva["Rend_acum"] = rend_acum[0:]
df_pasiva.head(15)

,Timestamp,capital,Rend,Rend_acum
0,2020-01-31,9.904000e+05,0.000000,1.154945
1,2020-02-29,9.259087e+05,-0.065116,1.079739
2,2020-03-31,7.755481e+05,-0.162392,0.904398
3,2020-04-30,8.224200e+05,0.060437,0.959057
4,2020-05-31,8.200611e+05,-0.002868,0.956306
5,2020-06-30,8.580205e+05,0.046288,1.000572
6,2020-07-31,8.482532e+05,-0.011384,0.989182
7,2020-08-31,8.464344e+05,-0.002144,0.987061
8,2020-09-30,8.602877e+05,0.016367,1.003216
9,2020-10-31,8.511331e+05,-0.010641,0.992541


In [33]:
#Inversión Activa
prices2= prices_A.iloc[0,:]

DFA = df
active_m1= DFA[0].reset_index(drop=True)
#Peso ponderado
peso_pondact= active_m1["Peso (%)"]/100
#Capital Invertido en T=0
peso_dinact= cap_i * peso_pondact


In [34]:
#Número de titulos exactos
acciones_act=prices_A.iloc[0, :]
titulos_act= [peso_dinact[i] / acciones_act[i] for i in range(0, len(acciones_act))]
n_titulos_act=[mt.trunc(titulos_act[i]) for i in range (0, len(titulos_act))]

fee_act= [sum((n_titulos_act[i] * acciones_act) *0.00125) for i in range(0,len(acciones_act))]


In [35]:
#Precio mensual por la cantidad de titulos
value_i_act= sum(n_titulos_act*acciones_act)

cash_act= cap_i - value_i_act - sum(fee_act)


In [36]:
#Valor del portafolio al primer mes
pvalue_actm1= value_i_act + cash_act

pvalue_actmr=[sum(prices_A.iloc[i, :] * titulos_act) for i in range (len(prices_A[0:]))]

In [38]:
#Mover los titulos cada mes
move=pd.DataFrame({'Close': acciones_act.iloc[0:], 'Rendimientos': prices2, 'Titulos': n_titulos_act[0:],
                            })
move.head(5)

,Close,Rendimientos,Titulos
Symbols,,,
AMXL.MX,14.346073,14.346073,9549
FEMSAUBD.MX,160.300339,160.300339,739
GFNORTEO.MX,105.844650,105.844650,1005
WALMEX.MX,53.542744,53.542744,1940
GMEXICOB.MX,42.370083,42.370083,1423


In [39]:
pd.options.mode.chained_assignment = None  # default='warn'
for i in range(1,len(move)):
    if move['Rendimientos'][i]<= -0.05:
        move['Titulos'][i]= move['Titulos'][i-1] +(move['Titulos'][i]*.05) // (move['Close'][i-1]*(1+0.00125))
    else:
        move['Titulos'][i]= move['Titulos'][i-1] - (move['Titulos'][i]*.05)

In [40]:
numtit_act= pvalue_actm1// acciones_act.iloc[0:]
pvalue_act= acciones_act.iloc[0:]*numtit_act
pvalue_act= move['Titulos']* move['Close']

In [48]:
df_activo= pd.DataFrame({"Timestamp": prices_A.index[0],"Capital":pvalue_act,"Rendimientos":pvalue_act.pct_change(),
                          "Rendimientos_Acumulados":pvalue_act/pvalue_act[0]})
df_activo.head(15)

,Timestamp,Capital,Rendimientos,Rendimientos_Acumulados
Symbols,,,,
AMXL.MX,2020-01-31,1.369907e+05,NaN,1.000000
FEMSAUBD.MX,2020-01-31,1.524777e+06,10.130517,11.130517
GFNORTEO.MX,2020-01-31,1.001396e+06,-0.343251,7.309960
WALMEX.MX,2020-01-31,5.013743e+05,-0.499325,3.659916
GMEXICOB.MX,2020-01-31,3.937028e+05,-0.214753,2.873939
CEMEXCPO.MX,2020-01-31,6.841576e+04,-0.826225,0.499419
TLEVISACPO.MX,2020-01-31,3.700821e+05,4.409311,2.701514
GAPB.MX,2020-01-31,1.935783e+06,4.230684,14.130765
ELEKTRA.MX,2020-01-31,1.223402e+07,5.319935,89.305514


In [49]:
#Históricos de las operaciones
df_operaciones= pd.DataFrame({'Timestamp': prices_A.index[0],'titulos_totales':move['Titulos'], 
                              'titulos_compra':move['Titulos'].diff(),'precios': move['Close']})
df_operaciones['Comisión']= move['Titulos'].diff()*move['Close']* 0.00125
df_operaciones['Comision_acum'] = df_operaciones['Comisión'].cumsum()
df_operaciones.head(10)

,Timestamp,titulos_totales,titulos_compra,precios,Comisión,Comision_acum
Symbols,,,,,,
AMXL.MX,2020-01-31,9549,NaN,14.346073,NaN,NaN
FEMSAUBD.MX,2020-01-31,9512,-37.0,160.300339,-7.413891,-7.413891
GFNORTEO.MX,2020-01-31,9461,-51.0,105.844650,-6.747596,-14.161487
WALMEX.MX,2020-01-31,9364,-97.0,53.542744,-6.492058,-20.653545
GMEXICOB.MX,2020-01-31,9292,-72.0,42.370083,-3.813307,-24.466852
CEMEXCPO.MX,2020-01-31,9008,-284.0,7.595000,-2.696225,-27.163077
TLEVISACPO.MX,2020-01-31,8960,-48.0,41.303806,-2.478228,-29.641306
GAPB.MX,2020-01-31,8952,-8.0,216.240250,-2.162402,-31.803708
ELEKTRA.MX,2020-01-31,8950,-2.0,1366.929688,-3.417324,-35.221032


In [45]:
rf = 0.0429/12
#Medidas de Atribución al Desempeño
df_medidas = pd.DataFrame({"medida":["rend_m","rend_c","sharpe"],
                           "descripción":['Rendimiento Promedio Mensual', 'Rendimiento Mensual Acumulado','Sharpe Ratio']})
df_medidas = df_medidas.set_index("medida")
df_medidas["inv_pasiva"] = (np.array(rend).mean(),np.array(rend_acum).mean(),(np.array(rend).mean()-rf)/np.array(rend_acum).std())
df_medidas["inv_activa"] = (prices2.mean(),prices2.mean(),(prices2.mean()-rf)/prices2.std())
df_medidas

,descripción,inv_pasiva,inv_activa
medida,,,
rend_m,Rendimiento Promedio Mensual,0.007197,120.590207
rend_c,Rendimiento Mensual Acumulado,1.250803,120.590207
sharpe,Sharpe Ratio,0.018183,0.511167
